<a href="https://colab.research.google.com/github/WRFitch/fyp/blob/main/src/fyp_preliminary_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup


*   Install & import necessary libraries
*   Set up Earth Engine datastores. 



In [ ]:
!pip install ipygee
!pip install fastai
!pip install fastai2

     |████████████████████████████████| 5.1MB 7.4MB/s 
     |████████████████████████████████| 133kB 53.9MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 225kB 44.0MB/s 
  Created wheel for ipygee: filename=ipygee-0.0.14-cp36-none-any.whl size=39897 sha256=daf1208ccfd039fc82ce8770ec9df05935ea60edcb72a12242a3df06e16b1847
  Stored in directory: /root/.cache/pip/wheels/57/f1/a2/ce9509a072ede592c0c9239ba66a1e9bb8af37bab3ca72fabc
  Created wheel for geetools: filename=geetools-0.6.7-cp36-none-any.whl size=86657 sha256=dd6294b9230fdf17c6a0221f66cec3a8b57fb36b2a817e1c776c2f6bf0d6c91c
  Stored in directory: /root/.cache/pip/wheels/ca/43/50/d2b86a80e0d9258a2bab669d205cebeeaac3662e58e0f18707
  Created wheel for pyshp: filename=pyshp-2.1.2-cp36-none-any.whl size=36217 sha256=dca83feea36eab00498ba719b6f950e00b716d00e02902d392f7aded80cd1cde
  Stored in directory: /root/.cache/pip/wheels/96/6c/53/4112475adf3b831da97f083163d0f38ee6daac9c1b13f7afea
Su

In [2]:
import ee
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from fastai.tabular import *
from IPython.display import Image

In [3]:
#sort earth engine setup
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=a430F8WU3feQ1cvfIKZKgUDbmA0JsXZ024ovJcM9qrs&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5XllvQvtNmTo5PT1qmye4ZrepP7z6fspX0ZWLK8LL5kzKdAXIwPp8

Successfully saved authorization token.


### Import the following datasets:

*   [Sentinel-2 Satellite photography](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)
*   [Sentinel-5 Precursor Data](https://developers.google.com/earth-engine/datasets/catalog/sentinel)
  *   [Aerosol](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_AER_AI)
  *   [Cloud](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_CLOUD)
  *   [Carbon Monoxide](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_CO)
  *   [Formaldehyde](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_HCHO)
  *   [Nitrogen Dioxide](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_NO2)
  *   [Ozone](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_O3)
  *   [Sulphur Dioxide](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_SO2)
  *   [Methane](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_CH4)
*   [ODIAC Fossil Fuel CO2 Emissions](https://db.cger.nies.go.jp/dataset/ODIAC/DL_odiac2019.html)


In [ ]:
# dataset must be imported in chunks to ensure the whole dataset can be worked through in this limited interpreter. 

In [4]:
# Import satellite photography dataset 
def maskS2clouds(image) :
  qa = image.select('QA60');

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

# Pre-filter to get less cloudy granules.
dataset = ee.ImageCollection('COPERNICUS/S2_SR') \
                  .filterDate('2020-01-01', '2020-01-30') \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                  .map(maskS2clouds)

In [11]:
# Downloads images from earth engine.

british_isles = ee.Geometry.Polygon(
        [[[-1.836112801004015, 59.808076330562756],
          [-8.779472176004015, 58.82140293049428],
          [-7.988456551004015, 55.71069203454839],
          [-11.196464363504015, 54.42753859549109],
          [-11.328300301004015, 50.967746003015044],
          [-9.526542488504015, 50.77361752815123],
          [-6.274589363504015, 51.81776248652293],
          [-5.395683113504015, 51.21615275310099],
          [-6.582206551004015, 49.56332371186494],
          [-3.110526863504015, 49.904165426606255],
          [1.240059073995985, 50.80139967619036],
          [2.426582511495985, 52.33095407387208],
          [1.767402823995985, 53.4183511305661],
          [0.5369340739959849, 53.44453305344514],
          [-1.616386238504015, 56.32474216074427],
          [-0.7814253010040151, 57.805828290000164]]])

test_bounds = ee.Geometry.Polygon(
        [[[-0.5558191148306624, 51.7509967841707],
          [-0.6354699937369124, 51.36850969263639],
          [-0.14108522811191238, 51.167448198751295],
          [0.3093542250131076, 51.30673830188811],
          [0.3944982679818576, 51.58576108751857],
          [0.0017370375131076088, 51.76629738492925]]])

#TODO move all this garbage over to the google earth engine ide, use that to export files into drive and use them directly in jupyter from there. 
def exportImg(image, bounds = test_bounds):
    Export.image.toDrive({
      image: image,
      description: 'imageToDriveExample',
      scale: 30,
      region: bounds
    })

def getImgs(bounds = test_bounds, m2_px = 10, dataset = None):
  print("getting images from ee")

  dataset = ee.ImageCollection('COPERNICUS/S2_SR') \
                  .filterDate('2020-01-01', '2020-01-30') \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                  .filterBounds(bounds) \
                  .map(maskS2clouds)
  dataset.map(exportImg)

In [12]:
getImgs(bounds = test_bounds)

getting images from ee


NameError: ignored

In [ ]:
from fastai.vision import *

path = untar_data(URLs.PETS)/'images'

pathlib.PosixPath

In [ ]:

type(get_image_files(path))

list